In [1]:
import pandas as pd
import numpy as np
import io
import math
import random
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
import cmapy
import nlpaug
import nlpaug.augmenter.audio as naa
from scipy.signal import butter, lfilter
import torch
from tqdm import tqdm
import os

In [2]:
audio_train=pd.read_csv('audio_data_train.csv')
audio_train.head(4)

,Unnamed: 0,Patient_ID,Disease,Recording_index,Chest_Location,Acquisition_Mode,Recording_equipment,Respiratory_cycles,Normal_cycles,Wheeze_cycles,Crackle_cycles,Both,Filename,Train/Test
0,3,103.0,NaN,2b2,Ar,mc,LittC2SE,6.0,2.0,4.0,0.0,0.0,103_2b2_Ar_mc_LittC2SE.txt,train
1,10,105.0,NaN,1b1,Tc,sc,Meditron,8.0,8.0,0.0,0.0,0.0,105_1b1_Tc_sc_Meditron.txt,train
2,11,106.0,NaN,2b1,Pl,mc,LittC2SE,9.0,1.0,0.0,8.0,0.0,106_2b1_Pl_mc_LittC2SE.txt,train
3,12,106.0,NaN,2b1,Pr,mc,LittC2SE,9.0,2.0,7.0,0.0,0.0,106_2b1_Pr_mc_LittC2SE.txt,train


In [3]:
sample_rate=4000
fs=30000
data_dir='AudioFiles2/'
desired_length=8
n_mels = 64
nfft = 256
hop = nfft//2
f_max = 2000
train_flag=1
lowcut=3000
highcut=5000

In [4]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b,a, data)
    return y

In [5]:
#slicing into breath cycles
def slice_data(start, end, raw_data, sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    y=butter_bandpass_filter(raw_data[start_ind: end_ind], lowcut, highcut, fs, order=5)
    return y

In [6]:
#getting label of each cycles
def get_label(crackle, wheeze):
    if crackle == 0 and wheeze == 0:
        return 0
    elif crackle == 1 and wheeze == 0:
        return 1
    elif crackle == 0 and wheeze == 1:
        return 2
    else:
        return 3

In [7]:
#getting breath cycle samples
def get_sound_samples(file_name, data_dir, sample_rate):
    sample_data = [file_name]
    data, rate = librosa.load(os.path.join(data_dir, file_name+'.wav'), sr=sample_rate)
    recording_annotations=pd.read_csv(data_dir+file_name+'.txt', sep='\t', header=None)
    
    for i in range(len(recording_annotations.index)):
        row = recording_annotations.loc[i]
        start = row[0]
        end = row[1]
        crackles = row[2]
        wheezes = row[3]
        audio_chunk = slice_data(start, end, data, rate)
        sample_data.append((audio_chunk, start,end, get_label(crackles, wheezes)))
    return sample_data

In [1]:
#creating list of breath cycles with respective labels
filenames=audio_train['Filename']
filenames_with_labels=[]
cycle_list = []
classwise_cycle_list = [[], [], [], []]
for idx, filename in tqdm(enumerate(filenames)):
            file_name=filename.split('.')[0]
            print(file_name+" going on "+str(idx)+" out of "+str(len(filenames)))
            data = get_sound_samples(file_name, data_dir, sample_rate)
            cycles_with_labels = [(d[0], d[3], file_name, cycle_idx, 0) for cycle_idx, d in enumerate(data[1:])]
            cycle_list.extend(cycles_with_labels)
            for cycle_idx, d in enumerate(cycles_with_labels):
                filenames_with_labels.append(file_name+'_'+str(d[3])+'_'+str(d[1]))
                classwise_cycle_list[d[1]].append(d)

NameError: name 'audio_train' is not defined

In [10]:
#splitting and padding cycles
def split_and_pad(original, desiredLength, sample_rate, types=0):
	if types==0:
		return split_and_pad_old(original, desiredLength, sample_rate)

	output_buffer_length = int(desiredLength*sample_rate)
	soundclip = original[0].copy()
	n_samples = len(soundclip)

	output = []
	if n_samples > output_buffer_length:
		frames = librosa.util.frame(soundclip, frame_length=output_buffer_length, hop_length=output_buffer_length//2, axis=0)
		for i in range(frames.shape[0]):
			output.append((frames[i], original[1], original[2], original[3], original[4], i, 0))

		last_id = frames.shape[0]*(output_buffer_length//2)
		last_sample = soundclip[last_id:]; pad_times = (output_buffer_length-len(last_sample))/len(last_sample)
		padded = generate_padded_samples(soundclip, last_sample, output_buffer_length, sample_rate, types)
		output.append((padded, original[1], original[2], original[3], original[4], i+1, pad_times))

	else:
		padded = generate_padded_samples(soundclip, soundclip, output_buffer_length, sample_rate, types); pad_times = (output_buffer_length-len(soundclip))/len(soundclip)
		output.append((padded, original[1], original[2], original[3], original[4], 0, pad_times))

	return output


def split_and_pad_old(original, desiredLength, sample_rate):
    output_buffer_length = int(desiredLength * sample_rate)
    soundclip = original[0].copy()
    n_samples = len(soundclip)
    total_length = n_samples / sample_rate #length of cycle in seconds
    n_slices = int(math.ceil(total_length / desiredLength)) #get the minimum number of slices needed
    samples_per_slice = n_samples // n_slices
    src_start = 0 #Staring index of the samples to copy from the original buffer
    output = [] #Holds the resultant slices
    for i in range(n_slices):
        src_end = min(src_start + samples_per_slice, n_samples)
        length = src_end - src_start
        copy = generate_padded_samples_old(soundclip[src_start:src_end], output_buffer_length)
        output.append((copy, original[1], original[2], original[3], original[4], i))
        src_start += length
    return output


def generate_padded_samples(original, source, output_length, sample_rate, types):
	copy = np.zeros(output_length, dtype=np.float32)
	src_length = len(source)
	left = output_length-src_length # amount to be padded
	# pad front or back
	prob = random.random()
	if types == 1:
		aug = original
	else:
		aug = gen_augmented(original, sample_rate)

	while len(aug) < left:
		aug = np.concatenate([aug, aug])

	if prob < 0.5:
		#pad back
		copy[left:] = source
		copy[:left] = aug[len(aug)-left:]
	else:
		#pad front
		copy[:src_length] = source[:]
		copy[src_length:] = aug[:left]

	return copy


In [11]:
#General augmentation augmentation
def gen_augmented(original, sample_rate):
	# list of augmentors available from the nlpaug library
	augment_list = [
	#naa.CropAug(sampling_rate=sample_rate)
	naa.NoiseAug(),
	naa.SpeedAug(),
	naa.LoudnessAug(factor=(0.5, 2)),
	naa.VtlpAug(sampling_rate=sample_rate, zone=(0.0, 1.0)),
	naa.PitchAug(sampling_rate=sample_rate, factor=(-1,3))
	]
	# sample augmentation randomly
	aug_idx = random.randint(0, len(augment_list)-1)
	augmented_data = augment_list[aug_idx].augment(original)
	return augmented_data

In [12]:
#creating sliced and padded data = audio_data
audio_data=[]
for idx, sample in enumerate(cycle_list):
            print(str(idx)+" out of "+str(len(cycle_list)))
            output = split_and_pad(sample, desired_length, sample_rate, types=1)
            audio_data.extend(output)

0 out of 4142
1 out of 4142
2 out of 4142
3 out of 4142
4 out of 4142
5 out of 4142
6 out of 4142
7 out of 4142
8 out of 4142
9 out of 4142
10 out of 4142
11 out of 4142
12 out of 4142
13 out of 4142
14 out of 4142
15 out of 4142
16 out of 4142
17 out of 4142
18 out of 4142
19 out of 4142
20 out of 4142
21 out of 4142
22 out of 4142
23 out of 4142
24 out of 4142
25 out of 4142
26 out of 4142
27 out of 4142
28 out of 4142
29 out of 4142
30 out of 4142
31 out of 4142
32 out of 4142
33 out of 4142
34 out of 4142
35 out of 4142
36 out of 4142
37 out of 4142
38 out of 4142
39 out of 4142
40 out of 4142
41 out of 4142
42 out of 4142
43 out of 4142
44 out of 4142
45 out of 4142
46 out of 4142
47 out of 4142
48 out of 4142
49 out of 4142
50 out of 4142
51 out of 4142
52 out of 4142
53 out of 4142
54 out of 4142
55 out of 4142
56 out of 4142
57 out of 4142
58 out of 4142
59 out of 4142
60 out of 4142
61 out of 4142
62 out of 4142
63 out of 4142
64 out of 4142
65 out of 4142
66 out of 4142
67 ou

604 out of 4142
605 out of 4142
606 out of 4142
607 out of 4142
608 out of 4142
609 out of 4142
610 out of 4142
611 out of 4142
612 out of 4142
613 out of 4142
614 out of 4142
615 out of 4142
616 out of 4142
617 out of 4142
618 out of 4142
619 out of 4142
620 out of 4142
621 out of 4142
622 out of 4142
623 out of 4142
624 out of 4142
625 out of 4142
626 out of 4142
627 out of 4142
628 out of 4142
629 out of 4142
630 out of 4142
631 out of 4142
632 out of 4142
633 out of 4142
634 out of 4142
635 out of 4142
636 out of 4142
637 out of 4142
638 out of 4142
639 out of 4142
640 out of 4142
641 out of 4142
642 out of 4142
643 out of 4142
644 out of 4142
645 out of 4142
646 out of 4142
647 out of 4142
648 out of 4142
649 out of 4142
650 out of 4142
651 out of 4142
652 out of 4142
653 out of 4142
654 out of 4142
655 out of 4142
656 out of 4142
657 out of 4142
658 out of 4142
659 out of 4142
660 out of 4142
661 out of 4142
662 out of 4142
663 out of 4142
664 out of 4142
665 out of 4142
666 out 

1247 out of 4142
1248 out of 4142
1249 out of 4142
1250 out of 4142
1251 out of 4142
1252 out of 4142
1253 out of 4142
1254 out of 4142
1255 out of 4142
1256 out of 4142
1257 out of 4142
1258 out of 4142
1259 out of 4142
1260 out of 4142
1261 out of 4142
1262 out of 4142
1263 out of 4142
1264 out of 4142
1265 out of 4142
1266 out of 4142
1267 out of 4142
1268 out of 4142
1269 out of 4142
1270 out of 4142
1271 out of 4142
1272 out of 4142
1273 out of 4142
1274 out of 4142
1275 out of 4142
1276 out of 4142
1277 out of 4142
1278 out of 4142
1279 out of 4142
1280 out of 4142
1281 out of 4142
1282 out of 4142
1283 out of 4142
1284 out of 4142
1285 out of 4142
1286 out of 4142
1287 out of 4142
1288 out of 4142
1289 out of 4142
1290 out of 4142
1291 out of 4142
1292 out of 4142
1293 out of 4142
1294 out of 4142
1295 out of 4142
1296 out of 4142
1297 out of 4142
1298 out of 4142
1299 out of 4142
1300 out of 4142
1301 out of 4142
1302 out of 4142
1303 out of 4142
1304 out of 4142
1305 out of 41

1884 out of 4142
1885 out of 4142
1886 out of 4142
1887 out of 4142
1888 out of 4142
1889 out of 4142
1890 out of 4142
1891 out of 4142
1892 out of 4142
1893 out of 4142
1894 out of 4142
1895 out of 4142
1896 out of 4142
1897 out of 4142
1898 out of 4142
1899 out of 4142
1900 out of 4142
1901 out of 4142
1902 out of 4142
1903 out of 4142
1904 out of 4142
1905 out of 4142
1906 out of 4142
1907 out of 4142
1908 out of 4142
1909 out of 4142
1910 out of 4142
1911 out of 4142
1912 out of 4142
1913 out of 4142
1914 out of 4142
1915 out of 4142
1916 out of 4142
1917 out of 4142
1918 out of 4142
1919 out of 4142
1920 out of 4142
1921 out of 4142
1922 out of 4142
1923 out of 4142
1924 out of 4142
1925 out of 4142
1926 out of 4142
1927 out of 4142
1928 out of 4142
1929 out of 4142
1930 out of 4142
1931 out of 4142
1932 out of 4142
1933 out of 4142
1934 out of 4142
1935 out of 4142
1936 out of 4142
1937 out of 4142
1938 out of 4142
1939 out of 4142
1940 out of 4142
1941 out of 4142
1942 out of 41

2518 out of 4142
2519 out of 4142
2520 out of 4142
2521 out of 4142
2522 out of 4142
2523 out of 4142
2524 out of 4142
2525 out of 4142
2526 out of 4142
2527 out of 4142
2528 out of 4142
2529 out of 4142
2530 out of 4142
2531 out of 4142
2532 out of 4142
2533 out of 4142
2534 out of 4142
2535 out of 4142
2536 out of 4142
2537 out of 4142
2538 out of 4142
2539 out of 4142
2540 out of 4142
2541 out of 4142
2542 out of 4142
2543 out of 4142
2544 out of 4142
2545 out of 4142
2546 out of 4142
2547 out of 4142
2548 out of 4142
2549 out of 4142
2550 out of 4142
2551 out of 4142
2552 out of 4142
2553 out of 4142
2554 out of 4142
2555 out of 4142
2556 out of 4142
2557 out of 4142
2558 out of 4142
2559 out of 4142
2560 out of 4142
2561 out of 4142
2562 out of 4142
2563 out of 4142
2564 out of 4142
2565 out of 4142
2566 out of 4142
2567 out of 4142
2568 out of 4142
2569 out of 4142
2570 out of 4142
2571 out of 4142
2572 out of 4142
2573 out of 4142
2574 out of 4142
2575 out of 4142
2576 out of 41

3118 out of 4142
3119 out of 4142
3120 out of 4142
3121 out of 4142
3122 out of 4142
3123 out of 4142
3124 out of 4142
3125 out of 4142
3126 out of 4142
3127 out of 4142
3128 out of 4142
3129 out of 4142
3130 out of 4142
3131 out of 4142
3132 out of 4142
3133 out of 4142
3134 out of 4142
3135 out of 4142
3136 out of 4142
3137 out of 4142
3138 out of 4142
3139 out of 4142
3140 out of 4142
3141 out of 4142
3142 out of 4142
3143 out of 4142
3144 out of 4142
3145 out of 4142
3146 out of 4142
3147 out of 4142
3148 out of 4142
3149 out of 4142
3150 out of 4142
3151 out of 4142
3152 out of 4142
3153 out of 4142
3154 out of 4142
3155 out of 4142
3156 out of 4142
3157 out of 4142
3158 out of 4142
3159 out of 4142
3160 out of 4142
3161 out of 4142
3162 out of 4142
3163 out of 4142
3164 out of 4142
3165 out of 4142
3166 out of 4142
3167 out of 4142
3168 out of 4142
3169 out of 4142
3170 out of 4142
3171 out of 4142
3172 out of 4142
3173 out of 4142
3174 out of 4142
3175 out of 4142
3176 out of 41

3621 out of 4142
3622 out of 4142
3623 out of 4142
3624 out of 4142
3625 out of 4142
3626 out of 4142
3627 out of 4142
3628 out of 4142
3629 out of 4142
3630 out of 4142
3631 out of 4142
3632 out of 4142
3633 out of 4142
3634 out of 4142
3635 out of 4142
3636 out of 4142
3637 out of 4142
3638 out of 4142
3639 out of 4142
3640 out of 4142
3641 out of 4142
3642 out of 4142
3643 out of 4142
3644 out of 4142
3645 out of 4142
3646 out of 4142
3647 out of 4142
3648 out of 4142
3649 out of 4142
3650 out of 4142
3651 out of 4142
3652 out of 4142
3653 out of 4142
3654 out of 4142
3655 out of 4142
3656 out of 4142
3657 out of 4142
3658 out of 4142
3659 out of 4142
3660 out of 4142
3661 out of 4142
3662 out of 4142
3663 out of 4142
3664 out of 4142
3665 out of 4142
3666 out of 4142
3667 out of 4142
3668 out of 4142
3669 out of 4142
3670 out of 4142
3671 out of 4142
3672 out of 4142
3673 out of 4142
3674 out of 4142
3675 out of 4142
3676 out of 4142
3677 out of 4142
3678 out of 4142
3679 out of 41

In [13]:
#Device_wise_list
file_to_device = {}
device_to_id = {}
device_id = 0
files = audio_train['Filename']#os.listdir(data_dir)
device_patient_list = []
pats = []
for f in files:
    device = f.strip().split('_')[-1].split('.')[0]
    if device not in device_to_id:
        device_to_id[device] = device_id
        device_id += 1
        device_patient_list.append([])
    file_to_device[f.strip().split('.')[0]] = device_to_id[device]
    pat = f.strip().split('_')[0]
    if pat not in device_patient_list[device_to_id[device]]:
        device_patient_list[device_to_id[device]].append(pat)
    if pat not in pats:
        pats.append(pat)

In [14]:
#creating labels for sliced and paddedd data = audio_data
class_probs=np.zeros(4)
identifiers=[]
device_wise=[]
labels=[]
for idx in range(device_id):
            device_wise.append([])
for idx, sample in enumerate(audio_data):
            class_probs[sample[1]] += 1.0
            labels.append(sample[1])
            identifiers.append(sample[2]+'_'+str(sample[3])+'_'+str(sample[1]))
            device_wise[file_to_device[sample[2]]].append(sample)

<b>audio_data</b> is the data which padded and augmented (in the simple way)

In [15]:
#creating mel spectrograms from augmented and padded data
def create_mel_raw(current_window, sample_rate, n_mels=128, f_min=50, f_max=4000, nfft=2048, hop=512, resz=1):
    S = librosa.feature.melspectrogram(y=current_window, sr=sample_rate, n_mels=n_mels, fmin=f_min, fmax=f_max, n_fft=nfft, hop_length=hop)
    S = librosa.power_to_db(S, ref=np.max)
    S = (S-S.min()) / (S.max() - S.min())
    S *= 255
    img = cv2.applyColorMap(S.astype(np.uint8), cmapy.cmap('magma'))
    height, width, _ = img.shape
    if resz > 0:
        img = cv2.resize(img, (width*resz, height*resz), interpolation=cv2.INTER_LINEAR)
    img = cv2.flip(img, 0)
    return img

In [18]:
train_dataset=[]
for index in range(len(audio_data)):
    audio=audio_data[index][0]
    # convert audio signal to spectrogram
    # spectrograms resized to 3x of original size
    print(str(index)+" out of "+str(len(audio_data)))
    audio_image = cv2.cvtColor(create_mel_raw(audio, sample_rate, f_max=f_max, 
            n_mels=n_mels, nfft=nfft, hop=hop, resz=3), cv2.COLOR_BGR2RGB)
    
    # blank region clipping
    audio_raw_gray = cv2.cvtColor(create_mel_raw(audio, sample_rate, f_max=f_max, 
            n_mels=n_mels, nfft=nfft, hop=hop), cv2.COLOR_BGR2GRAY)
    audio_raw_gray[audio_raw_gray < 10] = 0
    for row in range(audio_raw_gray.shape[0]):
        black_percent = len(np.where(audio_raw_gray[row,:]==0)[0])/len(audio_raw_gray[row,:])
        if black_percent < 0.80:
            break   
    if (row+1)*3 < audio_image.shape[0]:
        audio_image = audio_image[(row+1)*3:, :, :]
    audio_image = cv2.resize(audio_image, (audio_image.shape[1], n_mels*3), interpolation=cv2.INTER_LINEAR)
        
    
    label = audio_data[index][1]
    train_dataset.append((audio_image, label))

0 out of 4154
1 out of 4154
2 out of 4154
3 out of 4154
4 out of 4154
5 out of 4154
6 out of 4154
7 out of 4154
8 out of 4154
9 out of 4154
10 out of 4154
11 out of 4154
12 out of 4154
13 out of 4154
14 out of 4154
15 out of 4154
16 out of 4154
17 out of 4154
18 out of 4154
19 out of 4154
20 out of 4154
21 out of 4154
22 out of 4154
23 out of 4154
24 out of 4154
25 out of 4154
26 out of 4154
27 out of 4154
28 out of 4154
29 out of 4154
30 out of 4154
31 out of 4154
32 out of 4154
33 out of 4154
34 out of 4154
35 out of 4154
36 out of 4154
37 out of 4154
38 out of 4154
39 out of 4154
40 out of 4154
41 out of 4154
42 out of 4154
43 out of 4154
44 out of 4154
45 out of 4154
46 out of 4154
47 out of 4154
48 out of 4154
49 out of 4154
50 out of 4154
51 out of 4154
52 out of 4154
53 out of 4154
54 out of 4154
55 out of 4154
56 out of 4154
57 out of 4154
58 out of 4154
59 out of 4154
60 out of 4154
61 out of 4154
62 out of 4154
63 out of 4154
64 out of 4154
65 out of 4154
66 out of 4154
67 ou

520 out of 4154
521 out of 4154
522 out of 4154
523 out of 4154
524 out of 4154
525 out of 4154
526 out of 4154
527 out of 4154
528 out of 4154
529 out of 4154
530 out of 4154
531 out of 4154
532 out of 4154
533 out of 4154
534 out of 4154
535 out of 4154
536 out of 4154
537 out of 4154
538 out of 4154
539 out of 4154
540 out of 4154
541 out of 4154
542 out of 4154
543 out of 4154
544 out of 4154
545 out of 4154
546 out of 4154
547 out of 4154
548 out of 4154
549 out of 4154
550 out of 4154
551 out of 4154
552 out of 4154
553 out of 4154
554 out of 4154
555 out of 4154
556 out of 4154
557 out of 4154
558 out of 4154
559 out of 4154
560 out of 4154
561 out of 4154
562 out of 4154
563 out of 4154
564 out of 4154
565 out of 4154
566 out of 4154
567 out of 4154
568 out of 4154
569 out of 4154
570 out of 4154
571 out of 4154
572 out of 4154
573 out of 4154
574 out of 4154
575 out of 4154
576 out of 4154
577 out of 4154
578 out of 4154
579 out of 4154
580 out of 4154
581 out of 4154
582 out 

1035 out of 4154
1036 out of 4154
1037 out of 4154
1038 out of 4154
1039 out of 4154
1040 out of 4154
1041 out of 4154
1042 out of 4154
1043 out of 4154
1044 out of 4154
1045 out of 4154
1046 out of 4154
1047 out of 4154
1048 out of 4154
1049 out of 4154
1050 out of 4154
1051 out of 4154
1052 out of 4154
1053 out of 4154
1054 out of 4154
1055 out of 4154
1056 out of 4154
1057 out of 4154
1058 out of 4154
1059 out of 4154
1060 out of 4154
1061 out of 4154
1062 out of 4154
1063 out of 4154
1064 out of 4154
1065 out of 4154
1066 out of 4154
1067 out of 4154
1068 out of 4154
1069 out of 4154
1070 out of 4154
1071 out of 4154
1072 out of 4154
1073 out of 4154
1074 out of 4154
1075 out of 4154
1076 out of 4154
1077 out of 4154
1078 out of 4154
1079 out of 4154
1080 out of 4154
1081 out of 4154
1082 out of 4154
1083 out of 4154
1084 out of 4154
1085 out of 4154
1086 out of 4154
1087 out of 4154
1088 out of 4154
1089 out of 4154
1090 out of 4154
1091 out of 4154
1092 out of 4154
1093 out of 41

1522 out of 4154
1523 out of 4154
1524 out of 4154
1525 out of 4154
1526 out of 4154
1527 out of 4154
1528 out of 4154
1529 out of 4154
1530 out of 4154
1531 out of 4154
1532 out of 4154
1533 out of 4154
1534 out of 4154
1535 out of 4154
1536 out of 4154
1537 out of 4154
1538 out of 4154
1539 out of 4154
1540 out of 4154
1541 out of 4154
1542 out of 4154
1543 out of 4154
1544 out of 4154
1545 out of 4154
1546 out of 4154
1547 out of 4154
1548 out of 4154
1549 out of 4154
1550 out of 4154
1551 out of 4154
1552 out of 4154
1553 out of 4154
1554 out of 4154
1555 out of 4154
1556 out of 4154
1557 out of 4154
1558 out of 4154
1559 out of 4154
1560 out of 4154
1561 out of 4154
1562 out of 4154
1563 out of 4154
1564 out of 4154
1565 out of 4154
1566 out of 4154
1567 out of 4154
1568 out of 4154
1569 out of 4154
1570 out of 4154
1571 out of 4154
1572 out of 4154
1573 out of 4154
1574 out of 4154
1575 out of 4154
1576 out of 4154
1577 out of 4154
1578 out of 4154
1579 out of 4154
1580 out of 41

2004 out of 4154
2005 out of 4154
2006 out of 4154
2007 out of 4154
2008 out of 4154
2009 out of 4154
2010 out of 4154
2011 out of 4154
2012 out of 4154
2013 out of 4154
2014 out of 4154
2015 out of 4154
2016 out of 4154
2017 out of 4154
2018 out of 4154
2019 out of 4154
2020 out of 4154
2021 out of 4154
2022 out of 4154
2023 out of 4154
2024 out of 4154
2025 out of 4154
2026 out of 4154
2027 out of 4154
2028 out of 4154
2029 out of 4154
2030 out of 4154
2031 out of 4154
2032 out of 4154
2033 out of 4154
2034 out of 4154
2035 out of 4154
2036 out of 4154
2037 out of 4154
2038 out of 4154
2039 out of 4154
2040 out of 4154
2041 out of 4154
2042 out of 4154
2043 out of 4154
2044 out of 4154
2045 out of 4154
2046 out of 4154
2047 out of 4154
2048 out of 4154
2049 out of 4154
2050 out of 4154
2051 out of 4154
2052 out of 4154
2053 out of 4154
2054 out of 4154
2055 out of 4154
2056 out of 4154
2057 out of 4154
2058 out of 4154
2059 out of 4154
2060 out of 4154
2061 out of 4154
2062 out of 41

2508 out of 4154
2509 out of 4154
2510 out of 4154
2511 out of 4154
2512 out of 4154
2513 out of 4154
2514 out of 4154
2515 out of 4154
2516 out of 4154
2517 out of 4154
2518 out of 4154
2519 out of 4154
2520 out of 4154
2521 out of 4154
2522 out of 4154
2523 out of 4154
2524 out of 4154
2525 out of 4154
2526 out of 4154
2527 out of 4154
2528 out of 4154
2529 out of 4154
2530 out of 4154
2531 out of 4154
2532 out of 4154
2533 out of 4154
2534 out of 4154
2535 out of 4154
2536 out of 4154
2537 out of 4154
2538 out of 4154
2539 out of 4154
2540 out of 4154
2541 out of 4154
2542 out of 4154
2543 out of 4154
2544 out of 4154
2545 out of 4154
2546 out of 4154
2547 out of 4154
2548 out of 4154
2549 out of 4154
2550 out of 4154
2551 out of 4154
2552 out of 4154
2553 out of 4154
2554 out of 4154
2555 out of 4154
2556 out of 4154
2557 out of 4154
2558 out of 4154
2559 out of 4154
2560 out of 4154
2561 out of 4154
2562 out of 4154
2563 out of 4154
2564 out of 4154
2565 out of 4154
2566 out of 41

2996 out of 4154
2997 out of 4154
2998 out of 4154
2999 out of 4154
3000 out of 4154
3001 out of 4154
3002 out of 4154
3003 out of 4154
3004 out of 4154
3005 out of 4154
3006 out of 4154
3007 out of 4154
3008 out of 4154
3009 out of 4154
3010 out of 4154
3011 out of 4154
3012 out of 4154
3013 out of 4154
3014 out of 4154
3015 out of 4154
3016 out of 4154
3017 out of 4154
3018 out of 4154
3019 out of 4154
3020 out of 4154
3021 out of 4154
3022 out of 4154
3023 out of 4154
3024 out of 4154
3025 out of 4154
3026 out of 4154
3027 out of 4154
3028 out of 4154
3029 out of 4154
3030 out of 4154
3031 out of 4154
3032 out of 4154
3033 out of 4154
3034 out of 4154
3035 out of 4154
3036 out of 4154
3037 out of 4154
3038 out of 4154
3039 out of 4154
3040 out of 4154
3041 out of 4154
3042 out of 4154
3043 out of 4154
3044 out of 4154
3045 out of 4154
3046 out of 4154
3047 out of 4154
3048 out of 4154
3049 out of 4154
3050 out of 4154
3051 out of 4154
3052 out of 4154
3053 out of 4154
3054 out of 41

3486 out of 4154
3487 out of 4154
3488 out of 4154
3489 out of 4154
3490 out of 4154
3491 out of 4154
3492 out of 4154
3493 out of 4154
3494 out of 4154
3495 out of 4154
3496 out of 4154
3497 out of 4154
3498 out of 4154
3499 out of 4154
3500 out of 4154
3501 out of 4154
3502 out of 4154
3503 out of 4154
3504 out of 4154
3505 out of 4154
3506 out of 4154
3507 out of 4154
3508 out of 4154
3509 out of 4154
3510 out of 4154
3511 out of 4154
3512 out of 4154
3513 out of 4154
3514 out of 4154
3515 out of 4154
3516 out of 4154
3517 out of 4154
3518 out of 4154
3519 out of 4154
3520 out of 4154
3521 out of 4154
3522 out of 4154
3523 out of 4154
3524 out of 4154
3525 out of 4154
3526 out of 4154
3527 out of 4154
3528 out of 4154
3529 out of 4154
3530 out of 4154
3531 out of 4154
3532 out of 4154
3533 out of 4154
3534 out of 4154
3535 out of 4154
3536 out of 4154
3537 out of 4154
3538 out of 4154
3539 out of 4154
3540 out of 4154
3541 out of 4154
3542 out of 4154
3543 out of 4154
3544 out of 41

3977 out of 4154
3978 out of 4154
3979 out of 4154
3980 out of 4154
3981 out of 4154
3982 out of 4154
3983 out of 4154
3984 out of 4154
3985 out of 4154
3986 out of 4154
3987 out of 4154
3988 out of 4154
3989 out of 4154
3990 out of 4154
3991 out of 4154
3992 out of 4154
3993 out of 4154
3994 out of 4154
3995 out of 4154
3996 out of 4154
3997 out of 4154
3998 out of 4154
3999 out of 4154
4000 out of 4154
4001 out of 4154
4002 out of 4154
4003 out of 4154
4004 out of 4154
4005 out of 4154
4006 out of 4154
4007 out of 4154
4008 out of 4154
4009 out of 4154
4010 out of 4154
4011 out of 4154
4012 out of 4154
4013 out of 4154
4014 out of 4154
4015 out of 4154
4016 out of 4154
4017 out of 4154
4018 out of 4154
4019 out of 4154
4020 out of 4154
4021 out of 4154
4022 out of 4154
4023 out of 4154
4024 out of 4154
4025 out of 4154
4026 out of 4154
4027 out of 4154
4028 out of 4154
4029 out of 4154
4030 out of 4154
4031 out of 4154
4032 out of 4154
4033 out of 4154
4034 out of 4154
4035 out of 41

In [27]:
train_dataset[0][0].shape
len(train_dataset)

4154

In [30]:
X_train=[]
y_train=[]
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0])
    y_train.append(train_dataset[i][1])  
#display(X_train)
np.save('X_trainC', np.concatenate(X_train, axis=0).reshape(len(train_dataset), 192, 753, 3))
np.save('y_trainC', y_train)

# print("X_train length : "+str(len(X_train)))
# print("X_train sample length : "+str(X_train[0].shape))
# print("Array : "+str(np.concatenate(X_train, axis=0).shape))

# np.concatenate(X_train, axis=0).reshape(365, 192, 753, 3).shape